In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import os
import sys
import math
import pickle

In [3]:
import facenet

In [4]:
np.random.seed(666)

In [5]:
data_dir  = "/home/abaybektursun/projects/facenet/data/abay/aligned"
model_dir = "/home/abaybektursun/projects/facenet/data/20180402-114759/"

batch_size = 5
image_size = 160

classifier_filename = "recognition_model.pkl"

In [6]:
g = tf.Graph()

In [7]:
with g.as_default():
    with tf.Session() as sess:
        dataset = facenet.get_dataset(data_dir)
        paths, labels = facenet.get_image_paths_and_labels(dataset)
        # Restoe model
        facenet.load_model(model_dir)
        # Get input and output tensors
        images_placeholder = g.get_tensor_by_name("input:0")
        embeddings = g.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = g.get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]
        
        # Run forward pass to calculate embeddings
        nrof_images = len(paths)
        nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
        emb_array = np.zeros((nrof_images, embedding_size))

        for i in range(nrof_batches_per_epoch):
            start_index = i*batch_size
            end_index = min((i+1)*batch_size, nrof_images)

            paths_batch = paths[start_index:end_index]
            images = facenet.load_data(paths_batch, False, False, image_size)
            feed_dict = { images_placeholder:images, phase_train_placeholder:False }
            emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)

Model directory: /home/abaybektursun/projects/facenet/data/20180402-114759/
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from /home/abaybektursun/projects/facenet/data/20180402-114759/model-20180402-114759.ckpt-275


In [ ]:
len([n.name for n in g.as_graph_def().node])

In [ ]:
emb_array.shape

In [ ]:
len(labels)

In [8]:
from sklearn.svm import SVC

In [20]:
# Train classifier
model = SVC(kernel='linear', probability=True)
model.fit(emb_array, labels)

# Create a list of class names
class_names = [ cls.name.replace('_', ' ') for cls in dataset]

# Saving classifier model
with open(classifier_filename, 'wb') as outfile:
    pickle.dump((model, class_names), outfile)
predictions = model.predict_proba(emb_array)
best_class_indices = np.argmax(predictions, axis=1)

In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(labels, best_class_indices, target_names=class_names))

             precision    recall  f1-score   support

       abay       1.00      1.00      1.00         7
       alua       1.00      1.00      1.00         3
       fred       1.00      1.00      1.00         4
       mika       1.00      1.00      1.00        12
      saiyn       1.00      1.00      1.00         9

avg / total       1.00      1.00      1.00        35

